# Testing

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [10]:
im = plt.imread('MotorcycleChase/img/00001.jpg')

In [16]:
im.shape 

(720, 1280, 3)

In [ ]:
def data_transformation(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)/255
    img = tf.image.resize(img, (128,128))
    return (img , label)

In [ ]:
def create_dataset(image_path, batch_size=128):
    content = [(os.path.join(image_path,'cat',file), [1,0]) for file in os.listdir(os.path.join(image_path, 'cat'))]
    content += [(os.path.join(image_path,'dog',file), [0,1]) for file in os.listdir(os.path.join(image_path, 'dog'))]

    X_train, X_test, Y_train, Y_test = train_test_split([a for (a,_) in content],[b for (_,b) in content], test_size=0.2, random_state=22)
    
    train_dataset = tf.data.Dataset.from_tensor_slices((tf.constant(X_train), tf.constant(Y_train))).map(data_transformation, num_parallel_calls=tf.data.experimental.AUTOTUNE).cache().batch(batch_size)
    test_dataset  = tf.data.Dataset.from_tensor_slices((tf.constant(X_test), tf.constant(Y_test))).map(data_transformation, num_parallel_calls=tf.data.experimental.AUTOTUNE).cache().batch(batch_size)
    
    return train_dataset, test_dataset

In [ ]:
def get_model():
    model = keras.models.Sequential()
    model.add(layers.Conv2D(32,3,activation='relu',input_shape=(128,640,3))))
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(64,3,activation='relu')))
    model.add(layers.MaxPooling2D())
    model.add(layers.Flatten())
    model.add(layers.Dense(256,activation='relu')))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(128,activation='relu')))
    model.add(layers.Dropout(0.2))
    model.add(layers.Denese(32,activation='relu')))
    model.add(layers.Dropout(0.2))
    model.add(layers.Denese(4,activation='softmax')))
    model.compile(loss='categorical_crossentropy',optimizer ='adam', metrics=['accuracy', 'f1-score'])
    return model

In [ ]:
model = get_model()
model.summary()